# Latent Dirichlet Allocation pour des Données Textes

Dans ce TP 

* Appliquer des techniques de prétraitement standard sur les données textuelles Wikipedia en utilisant Pandas
* Utiliser  Gensim pour adapter un modèle d'allocation latente de Dirichlet (LDA)
* Explorer et interpréter les résultats, y compris les mots-clés de sujet et les assignations de sujets pour les documents


## Prétraitement des données texte


In [ ]:
import pandas
df = pandas.read_csv('people_wiki.csv')
df

In [26]:
Text = df.drop(columns=['URI', 'name'], axis=1)

In [ ]:
# Load the regular expression library
import re
# Remove punctuation
Text['text_processed'] = \
Text['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
Text['text_processed'] = \
Text['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
Text['text_processed'].head()

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(Text['text_processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = Text.text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
def explore_topic(topic_number, topn=50):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda_model.show_topic(topic_number, topn):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
explore_topic(1,5)

# Identifier lesTopiques par mots-clés




## Question 2: (2 pts)
Quelle est la somme des probabilités assignées aux 50 premiers mots du troisième topique?

In [2]:
#Tapez votre code ici







Regardons les 10 premiers mots pour chaque topique pour voir si nous pouvons identifier n'importe quels thèmes:

In [41]:
explore_topic(2, topn=10)

term                 frequency

league               0.012
played               0.012
season               0.012
football             0.009
club                 0.007
team                 0.007
first                0.007
coach                0.007
born                 0.006
university           0.005


Nous proposons les thèmes suivants pour chaque topique:

- topic 0: Music, TV, and film
- topic 1: Team sports
- topic 2: Art and publishing
- topic 3: American college and politics
- topic 4: International athletics
- topic 5: International music 
- topic 6: Science and research 
- topic 7: Business  
- topic 8: Great Britain and Australia  
- topic 9: General politics



In [11]:
themes = ['iMusic, TV, and film','Team sports','Art and publishing','American college and politics',' International athletics','International music', \
         'Science and research','Business','Great Britain and Australia','General politics']

# Distributions de topique pour certains exemples de documents


LDA se base sur le modèle mixte, ce qui signifie que chaque document peut partiellement appartenir à plusieurs topiques différents. Pour chaque document, l'appartenance à un topic est exprimée sous la forme d'un vecteur de poids égal à un; L'importance de chaque poids indique le degré auquel le document représente ce sujet particulier.

Nous allons explorer cela dans notre modèle  en examinant les distributions des topics pour quelques exemples d'articles Wikipedia de notre ensemble de données. Nous devrions trouver que ces articles ont les poids les plus élevés sur les topics dont les thèmes sont les plus important pour le topique de l'article - par exemple, nous nous attendons d'un article sur un homme de politique d'accorder un poids relativement élevé sur des topiques liés au gouvernement, un article sur un athlète devrait mettre plus de poids sur des topiques liés aux sports...


In [ ]:
obama = df [df['name'] == 'Barack Obama']
data1 = obama.text.values.tolist()
data_words1 = list(sent_to_words(data1))
# remove stop words
data_words1 = remove_stopwords(data_words1)
print(data_words1[:1][0][:30])

In [ ]:
   
id2word1 = corpora.Dictionary(data_words1)
# Create Corpus
texts1 = data_words1
# Term Document Frequency
corpus1 = [id2word.doc2bow(text) for text in texts1]
vect=lda_model[corpus1]

vect[0] 


## Question 3: (3 pts)

Quel est le topique le plus étroitement lié à l'article sur l'ancien président américain George W. Bush? 

In [ ]:
#Tapez votre code ici







In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    
LDAvis_prepared